In [1]:
import requests
import pandas as pd
import datetime
import time
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [2]:
# test games api https://api.squiggle.com.au/?q=games;year=2017;round=1
headers = {
    'User-Agent': 'AFLNaturalLanguageQueryBuilder/1.0 (tobyprofitt1@gmail.com)'
}
year = 2023
round = 28
r = requests.get('https://api.squiggle.com.au/?q=games;year={};round={}'.format(year, round), headers=headers)
games = r.json()
games_df = pd.DataFrame(games['games'])
games_df.head()

,localtime,tz,venue,ascore,hbehinds,complete,winner,id,round,agoals,ateam,hteamid,date,unixtime,hgoals,updated,is_grand_final,winnerteamid,ateamid,timestr,hteam,roundname,hscore,abehinds,is_final,year
0,2023-09-30 14:30:00,+10:00,M.C.G.,86,18,100,Collingwood,34682,28,13,Brisbane Lions,4,2023-09-30 14:30:00,1696048200,12,2023-09-30 17:21:43,1,4,2,Full Time,Collingwood,Grand Final,90,8,6,2023


In [3]:
def get_all_match_data(headers):
    data = {}
    current_year = datetime.datetime.now().year
    years = range(1996, current_year+1)
    rounds = range(1, 29)
    for year in tqdm(years):
        for round in rounds:
            r = requests.get('https://api.squiggle.com.au/?q=games;year={};round={}'.format(year, round), headers=headers)
            games = r.json()
            games_df = pd.DataFrame(games['games'])
            data['{}-{}'.format(year, round)] = games_df
            time.sleep(1)
    return data

headers = {
    'User-Agent': 'AFLNaturalLanguageQueryBuilder/1.0 (tobyprofitt1@gmail.com)'
}

data = get_all_match_data(headers)

  0%|          | 0/28 [00:00<?, ?it/s]

100%|██████████| 28/28 [18:31<00:00, 39.70s/it]


In [18]:
years = range(1996, 2024)
rounds = range(1, 29)
for year in years:
    # Stack each round of each year on top of each other
    df = pd.concat([data['{}-{}'.format(year, round)] for round in rounds])
        
    if not df.empty:
        df.to_csv('../data/matchinfo/{}.csv'.format(year), index=False)

C:\Users\tobyp\AppData\Local\Temp\ipykernel_21896\4005939245.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([data['{}-{}'.format(year, round)] for round in rounds])


## Now add to database

In [19]:
import sqlite3

conn = sqlite3.connect('../afl.db')

In [73]:
MATCHINFO = '''
CREATE TABLE game (
    round INTEGER,
    agoals INTEGER,
    id INTEGER PRIMARY KEY,
    unixtime INTEGER,
    date TEXT,
    hteamid INTEGER,
    ateam TEXT,
    tz TEXT,
    venue TEXT,
    localtime TEXT,
    winner TEXT,
    complete INTEGER,
    hbehinds INTEGER,
    ascore INTEGER,
    roundname TEXT,
    year INTEGER,
    is_final INTEGER,
    hscore INTEGER,
    abehinds INTEGER,
    is_grand_final INTEGER,
    updated TEXT,
    hgoals INTEGER,
    timestr TEXT,
    hteam TEXT,
    winnerteamid REAL,
    ateamid INTEGER
);
'''

conn.execute(MATCHINFO)

OperationalError: table game already exists

In [27]:
print(data['2023-28'].dtypes)
print(data['2023-28'].head(1))

winner            object
complete           int64
hteam             object
timestr           object
ateamid            int64
winnerteamid       int64
hbehinds           int64
ascore             int64
is_grand_final     int64
tz                object
venue             object
updated           object
hgoals             int64
localtime         object
year               int64
unixtime           int64
date              object
hteamid            int64
ateam             object
is_final           int64
hscore             int64
abehinds           int64
roundname         object
round              int64
agoals             int64
id                 int64
dtype: object
        winner  complete        hteam    timestr  ateamid  winnerteamid  \
0  Collingwood       100  Collingwood  Full Time        2             4   

   hbehinds  ascore  is_grand_final      tz   venue              updated  \
0        18      86               1  +10:00  M.C.G.  2023-09-30 17:21:43   

   hgoals            localtime  

In [46]:
# Prepare the data for insertion
data_to_insert = []
for key, df in data.items():
    for index, row in df.iterrows():
        data_to_insert.append(tuple(row))

errors = []

# Define the SQL query
insert_query = '''
INSERT INTO game (round, agoals, id, unixtime, date, hteamid, ateam, tz, venue, localtime, winner, complete, hbehinds, ascore, roundname, year, is_final, hscore, abehinds, is_grand_final, updated, hgoals, timestr, hteam, winnerteamid, ateamid)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

for i in range(1, len(data_to_insert)):
    try:
        # Execute the insertion
        conn.execute(insert_query, data_to_insert[i])
        # Commit the changes
        conn.commit()
    except Exception as e:
        print(e)
        errors.append(data_to_insert[i])
        conn.rollback()
        continue

# Execute the insertion
# conn.executemany(insert_query, data_to_insert)

# Commit the changes
# conn.commit()


datatype mismatch
datatype mismatch
datatype mismatch
datatype mismatch
datatype mismatch
datatype mismatch
datatype mismatch
datatype mismatch
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE constraint failed: game.id
UNIQUE

In [59]:
print(len(conn.execute('SELECT * FROM game').fetchall()))
print(len(errors))

476
4942


#### Why are there double game.ids?

In [67]:
# Stack dataframes on top of each other to each year, and add year column
years = range(1996, 2024)
rounds = range(1, 29)
df = pd.DataFrame()
for year in years:
    for round in rounds:
        df = pd.concat([df, data['{}-{}'.format(year, round)]])
        df['year'] = year

C:\Users\tobyp\AppData\Local\Temp\ipykernel_21896\2130340799.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, data['{}-{}'.format(year, round)]])


In [72]:
import sqlite3

conn = sqlite3.connect('../afl.db')

In [74]:
MATCHINFO = '''
CREATE TABLE game (
    round INTEGER,
    agoals INTEGER,
    id INTEGER PRIMARY KEY,
    unixtime INTEGER,
    date TEXT,
    hteamid INTEGER,
    ateam TEXT,
    tz TEXT,
    venue TEXT,
    localtime TEXT,
    winner TEXT,
    complete INTEGER,
    hbehinds INTEGER,
    ascore INTEGER,
    roundname TEXT,
    year INTEGER,
    is_final INTEGER,
    hscore INTEGER,
    abehinds INTEGER,
    is_grand_final INTEGER,
    updated TEXT,
    hgoals INTEGER,
    timestr TEXT,
    hteam TEXT,
    winnerteamid REAL,
    ateamid INTEGER
);
'''

conn.execute("DROP TABLE IF EXISTS game")

conn.execute(MATCHINFO)

In [75]:
# Stack dataframes on top of each other to each year, and add year column
years = range(1996, 2024)
rounds = range(1, 29)
df = pd.DataFrame()
for year in years:
    for round in rounds:
        df = pd.concat([df, data['{}-{}'.format(year, round)]])
        df['year'] = year

C:\Users\tobyp\AppData\Local\Temp\ipykernel_21896\2130340799.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, data['{}-{}'.format(year, round)]])


In [76]:
# Insert the data into the database
df.to_sql('game', conn, if_exists='append', index=False)

5419

In [77]:
# Check the data has been inserted
print(len(conn.execute('SELECT * FROM game').fetchall()))

5419
